Q.1   What is a Common Table Expression (CTE), and how does it improve SQL query readability?

...> A Common Table Expression (CTE) is a temporary, named result set in SQL that you can reference within a SELECT, INSERT, UPDATE, or DELETE statement.
It is defined using the WITH keyword and exists only for the duration of the query.

Syntax of a CTE
WITH cte_name AS (
    SELECT column1, column2
    FROM table_name
    WHERE condition
)
SELECT *
FROM cte_name;


Simple Example
Suppose you want employees who earn more than the average salary:
WITH AvgSalary AS (
    SELECT AVG(salary) AS avg_salary
    FROM employees
)
SELECT *
FROM employees
WHERE salary > (SELECT avg_salary FROM AvgSalary);


How CTEs Improve SQL Query Readability
1. Breaks Complex Queries into Logical Parts
Instead of writing long nested subqueries, CTEs allow you to separate logic into steps.
❌ Without CTE (harder to read):
SELECT *
FROM employees
WHERE salary > (
    SELECT AVG(salary)
    FROM employees
);

✅ With CTE (clear intent):
WITH AvgSalary AS (
    SELECT AVG(salary) AS avg_salary
    FROM employees
)
SELECT *
FROM employees
WHERE salary > (SELECT avg_salary FROM AvgSalary);


2. Improves Maintainability


Each CTE has a meaningful name


Easier to understand, modify, and debug


Changes can be made in one place without rewriting subqueries



3. Avoids Repeating Subqueries
You can reuse the same CTE multiple times within a query.
WITH HighEarners AS (
    SELECT *
    FROM employees
    WHERE salary > 60000
)
SELECT department, COUNT(*)
FROM HighEarners
GROUP BY department;


4. Supports Recursive Queries
CTEs are ideal for hierarchical data (e.g., managers and employees).
WITH RECURSIVE EmployeeHierarchy AS (
    SELECT emp_id, manager_id
    FROM employees
    WHERE manager_id IS NULL
    UNION ALL
    SELECT e.emp_id, e.manager_id
    FROM employees e
    JOIN EmployeeHierarchy eh ON e.manager_id = eh.emp_id
)
SELECT * FROM EmployeeHierarchy;


5. Cleaner Than Temporary Tables


No need to create and drop temp tables


Scoped only to the query


Makes scripts shorter and safer



Summary
A CTE:


Is a temporary named query result


Improves readability, clarity, and maintainability


Simplifies complex queries


Supports recursive data processing


👉 Think of a CTE as a readable building block for complex SQL logic.


Q.2 Why are some views updatable while others are read-only? Explain with an example.

...>> Why Views Can Be Updatable

A view is updatable when:

It is based on a single table

It contains direct column mappings (no calculations)

It does not use features that change row structure

In such cases, the database knows exactly which row and column in the base table should be updated.

Example of an Updatable View
Base Table
CREATE TABLE employees (
    emp_id INT PRIMARY KEY,
    name VARCHAR(50),
    salary INT,
    department VARCHAR(30)
);

Updatable View
CREATE VIEW emp_basic AS
SELECT emp_id, name, salary
FROM employees;

Update Through the View
UPDATE emp_basic
SET salary = 60000
WHERE emp_id = 101;


✅ This works because:

The view references one table

No aggregation, joins, or computed columns are used

Each row in the view corresponds to exactly one row in employees

Why Some Views Are Read-Only

A view becomes read-only when the database cannot reliably determine how to apply an update to the base tables.

Common Reasons a View Is Read-Only
1. Aggregate Functions
CREATE VIEW dept_avg_salary AS
SELECT department, AVG(salary) AS avg_salary
FROM employees
GROUP BY department;


❌ Not updatable
👉 You cannot update an average back into individual rows.

2. JOINs Between Multiple Tables
CREATE VIEW emp_dept AS
SELECT e.emp_id, e.name, d.dept_name
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id;


❌ Usually read-only
👉 Updating dept_name doesn’t clearly indicate which table or row to modify.

3. Derived or Calculated Columns
CREATE VIEW emp_bonus AS
SELECT emp_id, salary, salary * 0.10 AS bonus
FROM employees;


❌ Not updatable
👉 bonus does not exist in the base table.

4. DISTINCT Clause
CREATE VIEW unique_departments AS
SELECT DISTINCT department
FROM employees;


❌ Not updatable
👉 Duplicate rows are removed, so row mapping is unclear.

5. GROUP BY, HAVING, UNION

Any view using these clauses is read-only because rows no longer map 1-to-1 to base table rows.

Summary Table
Feature Used in View	Updatable?	Reason
Single table	✅ Yes	Clear row mapping
Aggregate functions	❌ No	No direct row mapping
JOINs	❌ Usually	Ambiguous table updates
Computed columns	❌ No	Column not stored
DISTINCT	❌ No	Row identity lost
Key Idea to Remember

🔑 A view is updatable only if each row in the view corresponds to exactly one row in a single base table.

If that one-to-one mapping is broken, the view becomes read-only.

Q.3 What advantages do stored procedures offer compared to writing raw SQL queries repeatedly?

..>> Stored procedures offer several important advantages over repeatedly writing raw SQL queries, especially in terms of performance, security, maintainability, and consistency.

1. Improved Performance

Stored procedures are precompiled and stored in the database.

Execution plans can be reused, reducing parsing and optimization time.

Less network traffic since only the procedure name and parameters are sent, not the full SQL.

Example:

CALL GetEmployeeByDept('IT');


instead of sending a long SELECT query every time.

2. Reusability

Write the logic once and reuse it across multiple applications or queries.

Ensures consistent business rules everywhere.

CREATE PROCEDURE GetHighSalaryEmployees()
BEGIN
    SELECT * FROM employees WHERE salary > 60000;
END;

3. Better Security

Users can be granted permission to execute a procedure without direct access to tables.

Helps prevent SQL injection attacks.

Sensitive logic and table structure are hidden.

GRANT EXECUTE ON PROCEDURE GetHighSalaryEmployees TO app_user;

4. Easier Maintenance

Logic changes are made in one place.

No need to modify multiple SQL queries across applications.

Instead of updating dozens of queries:

WHERE salary > 60000


Change it once inside the procedure.

5. Encapsulation of Business Logic

Business rules live in the database, not scattered across applications.

Ensures uniform validation and calculations.

Example:

CREATE PROCEDURE AddEmployee(
    IN emp_name VARCHAR(50),
    IN emp_salary INT
)
BEGIN
    IF emp_salary < 30000 THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Salary too low';
    END IF;

    INSERT INTO employees(name, salary)
    VALUES (emp_name, emp_salary);
END;

6. Reduced Code Duplication

Avoids repeating the same complex joins, filters, and calculations.

Cleaner application code.

7. Transaction Management

Stored procedures can control transactions (BEGIN, COMMIT, ROLLBACK).

Ensures data consistency in multi-step operations.

START TRANSACTION;
-- multiple statements
COMMIT;

8. Centralized Error Handling

Use exception handling within procedures.

Consistent error messages and behavior.

Summary Table
Advantage	Benefit
Performance	Faster execution, plan reuse
Reusability	Write once, use many times
Security	Limited table access, SQL injection prevention
Maintenance	Easier updates
Consistency	Same rules everywhere
Transactions	Better data integrity
✅ In short:

Stored procedures reduce repetition, improve performance and security, and make database logic easier to manage compared to writing raw SQL repeatedly.

Q.4 . What is the purpose of triggers in a database? Mention one use case where a trigger is essential.

...>> A trigger in a database is a special type of stored program that automatically executes (fires) in response to specific events on a table or view—such as INSERT, UPDATE, or DELETE.

Unlike stored procedures, triggers do not need to be explicitly called; they run automatically when the defined event occurs.

Purpose of Triggers

Triggers are mainly used to:

Enforce complex business rules

Rules that cannot be enforced using constraints alone.

Maintain data integrity

Automatically keep related tables consistent.

Audit and logging

Track changes to data (who changed what and when).

Automatic data validation

Prevent invalid operations.

Derived or calculated data maintenance

Automatically update dependent values.

Example Use Case Where a Trigger Is Essential
Automatic Audit Logging

Suppose you must keep a history of salary changes for employees for compliance or auditing.

Base Table
CREATE TABLE employees (
    emp_id INT PRIMARY KEY,
    name VARCHAR(50),
    salary INT
);

Audit Table
CREATE TABLE salary_audit (
    emp_id INT,
    old_salary INT,
    new_salary INT,
    changed_on TIMESTAMP
);

Trigger Example
CREATE TRIGGER salary_update_trigger
BEFORE UPDATE ON employees
FOR EACH ROW
BEGIN
    INSERT INTO salary_audit(emp_id, old_salary, new_salary, changed_on)
    VALUES (OLD.emp_id, OLD.salary, NEW.salary, NOW());
END;

What This Trigger Does

Fires automatically before a salary update

Stores old and new salary values

Records the date and time of change

🔹 Without a trigger, every application would have to remember to log changes manually—easy to forget and error-prone.

Why a Trigger Is Essential Here

Guaranteed execution (cannot be bypassed by applications)

Centralized auditing logic

Data compliance and traceability

Key Takeaway

👉 Triggers are used to automatically enforce rules or actions at the database level when data changes.
They are essential when an action must always occur whenever a specific event happens—such as audit logging, maintaining referential integrity, or preventing invalid updates.

Q.5 Explain the need for data modelling and normalization when designing a database.

..>> When designing a database, data modelling and normalization are essential to ensure the database is accurate, efficient, scalable, and easy to maintain.

1. Need for Data Modelling

Data modelling is the process of visually and logically designing how data is structured and how different data entities relate to each other.

Why Data Modelling Is Important
1. Clear Understanding of Business Requirements

Converts real-world requirements into structured data.

Identifies entities (tables), attributes (columns), and relationships.

Example:

Entity: Customer

Attributes: customer_id, name, email

2. Eliminates Ambiguity

Clearly defines what data is stored and how it is related.

Avoids inconsistent interpretations among developers and stakeholders.

3. Defines Relationships and Constraints

Establishes primary keys, foreign keys, cardinality, and rules.

Prevents orphan records and maintains referential integrity.

4. Improves Database Performance and Scalability

Helps design efficient queries and indexes.

Makes future expansion easier without breaking existing systems.

5. Acts as a Blueprint

Serves as a guide before actual database creation.

Reduces redesign and rework costs later.

2. Need for Normalization

Normalization is the process of organizing data to reduce redundancy and dependency by dividing tables and defining relationships.

Problems Without Normalization

Duplicate data stored in multiple places

Data inconsistency

Insert, update, and delete anomalies

Larger storage usage

Normal Forms (Brief Overview)
1. First Normal Form (1NF)

No repeating groups

Each field contains atomic values

❌ Not normalized:

OrderID | Products
101     | Laptop, Mouse


✅ 1NF:

OrderID | Product
101     | Laptop
101     | Mouse

2. Second Normal Form (2NF)

Must be in 1NF

No partial dependency on a composite key

3. Third Normal Form (3NF)

Must be in 2NF

No transitive dependency

Advantages of Normalization
1. Reduces Data Redundancy

Data stored only once

Saves storage space

2. Ensures Data Consistency

Updates happen in one place

Avoids conflicting values

3. Prevents Data Anomalies
Anomaly Type	Example
Insert	Cannot add customer without order
Update	Change address in multiple rows
Delete	Losing customer info when order is deleted
4. Improves Data Integrity

Clear relationships and constraints

Accurate and reliable data

5. Easier Maintenance

Smaller, well-structured tables

Changes affect fewer areas

Relationship Between Data Modelling and Normalization

Data modelling identifies entities and relationships.

Normalization refines the model to remove redundancy.

Together, they ensure a clean, logical, and efficient database design.

Final Summary

✔ Data modelling provides a clear structure and understanding of data
✔ Normalization removes redundancy and prevents anomalies
✔ Together, they ensure efficient, scalable, and reliable database systems

👉 A well-modeled and normalized database is the foundation of high-quality data and strong applications.

Q.6  Write a CTE to calculate the total revenue for each product
 (Revenues = Price × Quantity), and return only products where  revenue > 3000.

 ...>> Here’s a CTE-based SQL query that calculates total revenue per product and returns only products with revenue > 3000.

Assumption

You have a table (e.g., sales) with:

product_id or product_name

price

quantity

CTE Solution
WITH ProductRevenue AS (
    SELECT
        product_name,
        SUM(price * quantity) AS total_revenue
    FROM sales
    GROUP BY product_name
)
SELECT
    product_name,
    total_revenue
FROM ProductRevenue
WHERE total_revenue > 3000;

Explanation

CTE (ProductRevenue)

Calculates revenue using:

Revenue = Price × Quantity


Aggregates revenue per product using SUM() and GROUP BY.

Main Query

Filters products whose total revenue is greater than 3000.

Example Output
product_name	total_revenue
Laptop	4500
Mobile	7200
✅ Key Benefit of Using a CTE Here

Improves readability

Separates calculation logic from filtering logic

Easier to maintain and extend (e.g., adding tax or discounts later)

Q.7 Create a view named that shows:
 Category, TotalProducts, AveragePrice.

 ..>> Assumption

You have a table named products with columns:

category

price

product_id

SQL to Create the View
CREATE VIEW category_summary AS
SELECT
    category,
    COUNT(product_id) AS TotalProducts,
    AVG(price) AS AveragePrice
FROM products
GROUP BY category;

Explanation

COUNT(product_id) → Calculates the total number of products in each category

AVG(price) → Calculates the average price per category

GROUP BY category → Aggregates data category-wise

Example Output
Category	TotalProducts	AveragePrice
Electronics	10	45000
Clothing	25	1800
Furniture	8	32000
🔹 Note

This view is read-only because it uses:

Aggregate functions (COUNT, AVG)

GROUP BY

Q.8  Create an updatable view containing ProductID, ProductName, and Price.
 Then update the price productID = 1 using the view.

 ...>> Below is an example of creating an updatable view and then updating a product’s price using that view.

Assumption

You have a base table named products:

products
---------
ProductID
ProductName
Price
Category

1️⃣ Create an Updatable View

This view is updatable because:

It is based on a single table

No JOIN, GROUP BY, or aggregate functions

Columns map directly to the base table

CREATE VIEW product_view AS
SELECT
    ProductID,
    ProductName,
    Price
FROM products;

2️⃣ Update Price Using the View

Update the price of the product where ProductID = 1:

UPDATE product_view
SET Price = 5000
WHERE ProductID = 1;

✅ What Happens Internally

The update on product_view is automatically applied to the products table.

Since there is a one-to-one row mapping, the database knows exactly which row to update.

Key Point to Remember

🔹 Updatable views act like virtual tables—as long as they preserve a direct relationship with the base table, you can safely perform INSERT, UPDATE, and DELETE operations through them.

Q.9 . Create a stored procedure that accepts a category name and returns all products belonging to that
category.

...>> Below is an example of a stored procedure that accepts a category name as input and returns all products belonging to that category.

Assumption

You have a table named products with columns:

ProductID

ProductName

Price

Category

Stored Procedure (MySQL)
DELIMITER $$

CREATE PROCEDURE GetProductsByCategory (
    IN p_category VARCHAR(50)
)
BEGIN
    SELECT
        ProductID,
        ProductName,
        Price,
        Category
    FROM products
    WHERE Category = p_category;
END $$

DELIMITER ;

How to Call the Stored Procedure
CALL GetProductsByCategory('Electronics');

Explanation

IN p_category → Input parameter for category name

The SELECT statement filters products based on the given category

Returns all matching product records

Example Output
ProductID	ProductName	Price	Category
1	Laptop	55000	Electronics
3	Mobile	30000	Electronics
✅ Key Benefits

Reusable logic

Cleaner application code

Consistent filtering across queries.

Q.10 Create an AFTER DELETE trigger on the table that archives deleted product rows into a new
table . The archive should store ProductID, ProductName, Category, Price, and Deleted At timestamp.

..>> Below is a MySQL example that creates an AFTER DELETE trigger to archive deleted product records.

1️⃣ Create the Archive Table
CREATE TABLE products_archive (
    ProductID   INT,
    ProductName VARCHAR(100),
    Category    VARCHAR(50),
    Price       DECIMAL(10,2),
    DeletedAt   TIMESTAMP
);

2️⃣ Create the AFTER DELETE Trigger
DELIMITER $$

CREATE TRIGGER after_product_delete
AFTER DELETE ON products
FOR EACH ROW
BEGIN
    INSERT INTO products_archive (
        ProductID,
        ProductName,
        Category,
        Price,
        DeletedAt
    )
    VALUES (
        OLD.ProductID,
        OLD.ProductName,
        OLD.Category,
        OLD.Price,
        NOW()
    );
END $$

DELIMITER ;

3️⃣ How It Works

The trigger fires after a row is deleted from the products table

OLD holds the deleted row’s values

The deleted product is copied into products_archive

DeletedAt records the deletion time

✅ Key Use Case

This trigger is essential for:

Data recovery

Auditing and compliance

Tracking accidental deletions

🔹 Notes

This trigger runs automatically for every DELETE

It cannot be bypassed by applications

Works even for multi-row deletes.

